# Data analysis

This notebooks is used to analyse bunch electrons in a Plasma Wakefield simulation.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import openpmd_api as io
from scipy import constants, stats
from scipy.optimize import curve_fit

from matplotlib.colors import LogNorm, SymLogNorm, Normalize, hsv_to_rgb

In [ ]:
#path="/bigdata/hplsim/production/wrobel45/PWFA-bunch-bachelor1/test/simOutput/openPMD/simData_beta_%T.bp"
path="/bigdata/hplsim/production/wrobel45/PWFA-bunch-bachelor1/square_250MeV/030_square_bachelor1/simOutput/openPMD/simData_beta_%T.bp"
path_field="/bigdata/hplsim/production/wrobel45/PWFA-bunch-bachelor1/square_250MeV/030_square_bachelor1/simOutput/openPMD/simData_%T.bp"

## Definitions

Define different functions for data fitting.

In [ ]:
def window(t):
    """Calculates width of viewing window."""
    y_cells = 2048
    z_cells = 1024      # should be same as x_cells
    y_gpus = 8
    cellwidth = 0.5 * 0.1772e-6 * 1e6   # in micorometer, same for all directions
    Delta_t = 1.706e-16/1.28631
    windowwidth = y_cells * (y_gpus-1)/y_gpus * cellwidth
    windowheight = z_cells * cellwidth

    y_min = Delta_t * constants.c * t * 1e6 + windowwidth * 0.52
    y_max = Delta_t * constants.c * t * 1e6 + windowwidth * 0.72
    z_min = -windowheight / 2
    z_max =  windowheight / 2
    
    return y_min, y_max, z_min, z_max

In [ ]:
def Gauss(x, a, b, c):
    y = a * np.exp(-(x-b)**2 / (2*c**2))
    return y

def Gauss_dens(x, a, b):
    y = 1/np.sqrt(2*np.pi * b**2) * np.exp(-(x-a)**2 / (2*b**2))
    return y

def Gauss2(x, a1, b1, c1, a2, b2, c2):
    y = a1 * np.exp(-(x-b1)**2 / (2*c1**2)) + a2 * np.exp(-(x-b2)**2 / (2*c2**2))
    return y

def Gauss3(x, a1, b1, c1):
    y = c1 * np.exp(-(x-a1)**2 / (2*b1**2))
    return y

def Lorentz(x, a, b):
    y = 2/np.pi * b / (4*(x - a)**2 + b)
    return y

def Max_Boltz_2D(x, a):
    y = x/a * np.exp(-x**2 / (2*a))
    return y

In [ ]:
class particlePMD:
    def __init__(self, particle):
        x_off = particle["positionOffset"]["x"][:]
        x_off_unit = particle["positionOffset"]["x"].unit_SI
        y_off = particle["positionOffset"]["y"][:]
        y_off_unit = particle["positionOffset"]["y"].unit_SI
        z_off = particle["positionOffset"]["z"][:]
        z_off_unit = particle["positionOffset"]["z"].unit_SI

        x = particle["position"]["x"][:]
        x_unit = particle["position"]["x"].unit_SI
        y = particle["position"]["y"][:]
        y_unit = particle["position"]["y"].unit_SI
        z = particle["position"]["z"][:]
        z_unit = particle["position"]["z"].unit_SI

        p_x = particle["momentum"]["x"][:]
        p_x_unit = particle["momentum"]["x"].unit_SI
        p_y = particle["momentum"]["y"][:]
        p_y_unit = particle["momentum"]["y"].unit_SI
        p_z = particle["momentum"]["z"][:]
        p_z_unit = particle["momentum"]["z"].unit_SI

        x_probeE = particle['probeE']['x'][:]
        x_probeE_unit = particle["probeE"]["x"].unit_SI 
        y_probeE = particle['probeE']['y'][:]
        y_probeE_unit = particle["probeE"]["y"].unit_SI
        z_probeE = particle['probeE']['z'][:]
        z_probeE_unit = particle["probeE"]["z"].unit_SI

        x_probeB = particle['probeB']['x'][:]
        x_probeB_unit = particle["probeB"]["x"].unit_SI 
        y_probeB = particle['probeB']['y'][:]
        y_probeB_unit = particle["probeB"]["y"].unit_SI
        z_probeB = particle['probeB']['z'][:]
        z_probeB_unit = particle["probeB"]["z"].unit_SI

        particleId = particle["id"][io.Mesh_Record_Component.SCALAR][:]
        weight = particle["weighting"][io.Mesh_Record_Component.SCALAR][:]

        series.flush()

        self.x_pos = (x * x_unit + (x_off-512) * x_off_unit) * 1e6
        self.y_pos = (y * y_unit +  y_off      * y_off_unit) * 1e6
        self.z_pos = (z * z_unit + (z_off-512) * z_off_unit) * 1e6
        
        self.x_mom = p_x * p_x_unit
        self.y_mom = p_y * p_y_unit
        self.z_mom = p_z * p_z_unit
        
        self.x_probeE = x_probeE * x_probeE_unit
        self.y_probeE = y_probeE * y_probeE_unit
        self.z_probeE = z_probeE * z_probeE_unit
        
        self.x_probeB = x_probeB * x_probeB_unit
        self.y_probeB = y_probeB * y_probeB_unit
        self.z_probeB = z_probeB * z_probeB_unit
        
        self.particleId = particleId
        self.weight = np.float64(weight)
        
    def Energy(self):
        # squared total momentum
        p2_total = np.float64(self.x_mom**2 + self.y_mom**2 + self.z_mom**2)
        # energy per macro particle in Joule
        E_J = (constants.c * np.sqrt((constants.m_e*self.weight)**2 * constants.c**2 + p2_total) - (constants.m_e*self.weight) * constants.c**2) * 1/self.weight
        # conversion to MeV
        E = E_J / constants.e / 1e6
        
        return E
        
    def LorentzForce(self):
        x_vel = np.sqrt((self.x_mom / (constants.m_e*self.weight))**2 / (1 + (self.x_mom**2 + self.y_mom**2 + self.z_mom**2) / (constants.m_e*self.weight)**2 / constants.c**2))
        y_vel = np.sqrt((self.y_mom / (constants.m_e*self.weight))**2 / (1 + (self.x_mom**2 + self.y_mom**2 + self.z_mom**2) / (constants.m_e*self.weight)**2 / constants.c**2))
        z_vel = np.sqrt((self.z_mom / (constants.m_e*self.weight))**2 / (1 + (self.x_mom**2 + self.y_mom**2 + self.z_mom**2) / (constants.m_e*self.weight)**2 / constants.c**2))

        F_x_B = y_vel*self.z_probeB - z_vel*self.y_probeB
        F_y_B = z_vel*self.x_probeB - x_vel*self.z_probeB
        F_z_B = x_vel*self.y_probeB - y_vel*self.x_probeB

        F_x_L = -constants.e*self.weight * (self.x_probeE + F_x_B) * 1e3
        F_y_L = -constants.e*self.weight * (self.y_probeE + F_y_B) * 1e3
        F_z_L = -constants.e*self.weight * (self.z_probeE + F_z_B) * 1e3   
        
        return F_x_L, F_y_L, F_z_L

In [ ]:
def binned_to_grid(bin_x, bin_y):
    """Transforms 2D bins into grid.
    The left borders of the bins are taken as gridpoints, the rightmost border is discarded.
    """
    x_width = bin_x[1] - bin_x[0]
    y_width = bin_y[1] - bin_y[0]
    x = bin_x[:-1] + x_width/2
    y = bin_y[:-1] + y_width/2
    
    return x, y

## Density fields

Plots the charge- and energy-density fields for b- and e-Particles

In [ ]:
# Test for density plots
time = range(20000, 170000+1, 4000)
series = io.Series(path_field, io.Access.read_only)

for i, t in enumerate(time):
    y_min, y_max, z_min, z_max = window(t)
    e_field_charge = series.iterations[t].meshes["e_all_chargeDensity"][io.Mesh_Record_Component.SCALAR][:,:,512]
    e_field_energy = series.iterations[t].meshes["e_all_energyDensity"][io.Mesh_Record_Component.SCALAR][:,:,512]
    b_field_charge = series.iterations[t].meshes["b_all_chargeDensity"][io.Mesh_Record_Component.SCALAR][:,:,512]
    b_field_energy = series.iterations[t].meshes["b_all_energyDensity"][io.Mesh_Record_Component.SCALAR][:,:,512]
    series.flush()
    
    print("e_field_charge: ", np.min(e_field_charge), np.max(e_field_charge))
    print("e_field_energy: ", np.min(e_field_energy), np.max(e_field_energy))
    print("b_field_charge: ", np.min(b_field_charge), np.max(b_field_charge))
    print("b_field_energy: ", np.min(b_field_energy), np.max(b_field_energy))
    
    plt.figure(figsize=(16, 6), dpi=90)

    plt.subplot(121)
    plt.imshow(-e_field_charge, cmap='hot_r', norm=LogNorm(vmin=1e-4, vmax=20))
    plt.colorbar(label="electrons")    
    plt.imshow(-b_field_charge, cmap='Greens', norm=LogNorm(vmin=1e-12, vmax=3))
    plt.colorbar(label="bunch") 
    plt.xlabel("y")
    plt.ylabel("z")
    plt.title("charge density, timestep = {}".format(t))
    
    plt.subplot(122)
    plt.imshow(e_field_energy, cmap='hot_r', norm=LogNorm(vmin=1e-10, vmax=1e2))
    plt.colorbar(label="electrons")    
    plt.imshow(b_field_energy, cmap='Greens', norm=LogNorm(vmin=1e-5, vmax=1e3))
    plt.colorbar(label="bunch")    
    plt.xlabel("y")
    plt.ylabel("z")
    plt.title("energy density, timestep = {}".format(t))
    
    plt.show()
    #plt.savefig(fname="temp", facecolor="white")

del series


In [ ]:
# Test for density plots
time = range(20000, 170000+1, 4000)
series = io.Series(path_field, io.Access.read_only)

for i, t in enumerate(time):
    y_min, y_max, z_min, z_max = window(t)

    E_field = series.iterations[t].meshes["E"]["y"][:,:,512]
    b_field_charge = series.iterations[t].meshes["b_all_chargeDensity"][io.Mesh_Record_Component.SCALAR][:,:,512]
    series.flush()
    
    print("E_field: ", np.min(E_field), np.max(E_field))
    print("b_field_charge: ", np.min(b_field_charge), np.max(b_field_charge))
    
    plt.figure(figsize=(8, 6), dpi=70)

    plt.imshow(E_field, cmap='Spectral', vmin=-2e-2, vmax=2e-2, extent=(y_min, y_max, z_min, z_max), aspect='auto', origin="lower")
    plt.colorbar(label="E-field")    
    plt.imshow(-b_field_charge, cmap='Greens', norm=LogNorm(vmin=1e-9, vmax=3), extent=(y_min, y_max, z_min, z_max), aspect='auto', origin="lower")
    plt.colorbar(label="bunch") 
    plt.xlabel("y")
    plt.ylabel("z")
    plt.title("charge density, timestep = {}".format(t))
    
    plt.show()
    #plt.savefig(fname="temp", facecolor="white")

del series


## Phasespace

Plots the y-p_y (or other combinations) phase space.

In [ ]:
time = range(60000, 90000+1, 2000)
series = io.Series(path, io.Access.read_only)

for i, t in enumerate(time):
    par = particlePMD(series.iterations[t].particles["b_all"])
    y_min, y_max, z_min, z_max = window(t)

    # phase histogram
    plt.figure(figsize=(8, 6), dpi=90)
    plt.hist2d(par.y_pos, par.y_mom, bins=200, weights=par.weight, cmap="hot_r", range=[[y_min, y_max], [0, 7e-15]])
    plt.colorbar()
    plt.xlabel("$y \, \mathrm{[\mu m]}$")
    plt.ylabel("$p_y\, \mathrm{[Ns]}$")
    plt.title("y-p_y-phase space, timestep = {}".format(t))
    plt.show()
    #plt.savefig('temp.png', facecolor='w')

del series

## Energy-Divergence Histograms

In [ ]:
time = range(20000, 170000+1, 2000)
E_bin_num = 100                    # Bin number for energy histogram
peak_energy = 244.5             # peak energy for masks
div_max = 90                    # maximal range of viewed divergence

# energy-divergence histogram
range_E = [0, 400]
range_div = [-div_max, div_max]
bins1 = [1000, 201]
 
dE = (range_E[1] - range_E[0]) / bins1[0]
ddiv = (range_div[1] - range_div[0]) / bins1[1]
dQ = constants.e * 1e12

# divergence map
bins2 = 201

# divergence fit
# calculates index of given energy when changing the xbins or xrange
ind_Estart = int(240/dE)
ind_Eend = int(250/dE)
p1 = [10, 0, 3]


E_peak = np.zeros(len(time))
E_peak_stat = np.zeros(len(time))
E_peak_sys = np.zeros(len(time))
E_max = np.zeros(len(time))
E_min = np.zeros(len(time))
E_mean = np.zeros(len(time))
E_std = np.zeros(len(time))
E_hist = np.zeros((len(time), E_bin_num))    # stores energy histogram for every timestep

# return maximal energy over all timesteps
series = io.Series(path, io.Access.read_only)
par_end = particlePMD(series.iterations[time[-1]].particles["b_all"])
E_end = par_end.Energy()
max_Energy = np.max(E_end) + 5

for i, t in enumerate(time):
    par = particlePMD(series.iterations[t].particles["b_all"])
    y_min, y_max, z_min, z_max = window(t)

    E = par.Energy()

    E_max[i] = np.max(E)
    E_min[i] = np.min(E)
    E_mean[i] = np.mean(E)
    E_std[i] = np.std(E)
    E_hist[i][:] = np.histogram(E, weights=par.weight, bins=E_bin_num, range=(0, max_Energy))[0] * dQ / dE

    # divergence (or whatever it's actually called)
    div = np.arctan2(par.z_mom, par.y_mom)*1e3     # should be x_mom
    
    
    # energy-divergence histogram
    hist, edges_E, edges_div= np.histogram2d(E, div, weights=par.weight, bins=bins1, range=[range_E, range_div])
    hist *= dQ / dE / ddiv


    # divergence map
    dy = (y_max - y_min) / bins2
    dz = (z_max - z_min) / bins2
    statistic_div = stats.binned_statistic_2d(par.y_pos, par.z_pos, (np.abs(div)), statistic='mean', bins=bins2, range=[[y_min, y_max], [z_min, z_max]])[0]
    statistic_div *= 1 / dy / dz


    # energy fit
    pos_E = edges_E[:-1] + dE/2
    height_E = np.sum(hist, axis=1)*ddiv
    mask = (pos_E>180)
    # fitting to Gauss curve
    p0 = [15, 245, 6, 10, 210, 20]
    parameters, covariance = curve_fit(Gauss2, pos_E[mask], height_E[mask], p0)
    fit_a1, fit_b1, fit_c1, fit_a2, fit_b2, fit_c2 = parameters
    fit_E = Gauss2(pos_E, fit_a1, fit_b1, fit_c1, fit_a2, fit_b2, fit_c2)
    
    print("Energy fit:\n  Peak 1:     Height: {:.2f}, Mean: {:.2f} MeV, Std. Dev.: {:.2f} MeV\n  Peak 2:     Height: {:.2f}, Mean: {:.2f} MeV, Std. Dev.: {:.2f} MeV".format(fit_a1, fit_b1, fit_c1, fit_a2, fit_b2, fit_c2))
    E_peak[i] = fit_b1
    E_peak_stat[i] = np.sqrt(covariance[1,1])
    E_peak_sys[i] = dE


    # divergence fit
    pos_div = edges_div[:-1] + ddiv/2
    height_div = np.sum(hist[ind_Estart:ind_Eend,:], axis=0)*dE

    # fitting to Gauss curve
    parameters, covariance = curve_fit(Gauss, pos_div, height_div, p1)
    fit_a, fit_b, fit_c = parameters
    fit_y = Gauss(pos_div, fit_a, fit_b, fit_c)
    print("Divergence fit:\n  Height: {:.2f}, Mean: {:.2f} mrad, Std. Dev.: {:.2f} mrad".format(fit_a, fit_b, fit_c))

    plot = True
    if plot:
        plt.figure(figsize=(16, 10), dpi=70)

        # energy-divergence plot
        plt.subplot(221)
        plt.imshow(hist.T, origin="lower", interpolation='none', aspect='auto', cmap="hot_r", extent=(range_E[0], range_E[1], range_div[0], range_div[1]), norm=LogNorm(vmin=5e-3))
        plt.colorbar(label="$dQ/dE/d\\theta \, \mathrm{[pC/Mev/mrad]}$")
        plt.xlabel("Energy [MeV]")
        plt.ylabel("Divergence [mrad]")
        plt.title("t: {:1d}".format(t))

        plt.subplot(222)
        plt.imshow(statistic_div.T, cmap='magma_r', extent=(y_min, y_max, z_min, z_max), aspect='auto', origin="lower", interpolation='none')
        plt.colorbar(label='$d\\theta/dy/dz \, \mathrm{[mrad/\mu m^2]}$')
        plt.xlabel("$y \, \mathrm{[\mu m]}$")
        plt.ylabel("$z \, \mathrm{[\mu m]}$")
        plt.title("absolute divergence, timstep: {:1d}".format(t))

        plt.subplot(223)
        plt.bar(pos_E, height_E, label="data")
        plt.plot(pos_E[mask], fit_E[mask], label="fit", c='orange')
        plt.xlabel("Energy [MeV]")
        plt.ylabel("$dQ/dE \, \mathrm{[pC/Mev]}$")
        plt.xlim([0,400])
        plt.legend()
        plt.title("Energy count, t: {:1d}, |div| < {:.1f} mrad".format(t, div_max))

        plt.subplot(224)
        plt.bar(pos_div, height_div, label='data',width=0.5)
        plt.plot(pos_div, fit_y, label='fit', c='orange')
        plt.xlabel("Divergence [mrad]")
        plt.ylabel("$dQ/d\\theta \, \mathrm{[pC/mrad]}$")
        plt.xlim([-40,40])
        plt.title("250 MeV > energy > 240 MeV, t: {:1d}".format(t))
        plt.legend()

        #plt.savefig(fname="temp", facecolor="white")
        plt.show()
    else:
        print("timestep: ", t, "\n")

del series

In [ ]:
plt.figure(figsize=(16, 4), dpi=90)

plt.subplot(121)
plt.imshow(E_hist.T, origin='lower', extent=(time[0]-1e3, time[-1]+1e3, 0, max_Energy), aspect='auto', interpolation='none')
plt.plot(time, E_max, marker='.', label='$E_{max}$')
plt.errorbar(time, E_mean, yerr=E_std, marker='.', label='$E_{mean}$', capsize=3)
plt.plot(time, E_peak, marker='.', label='$E_{peak}$', c='red')
plt.plot(time, E_min, marker='.', label='$E_{min}$')
plt.colorbar(label='$Q/E \, \mathrm{[pC/MeV]}$')     # korrekte Einheit?
plt.xlabel("timestep")
plt.ylabel("energy [MeV]")
plt.title("energy over time")
plt.legend()

plt.subplot(122)
plt.errorbar(time, E_peak, yerr=E_peak_stat, marker='.', capsize=3, label="data")
plt.plot(time, E_peak+E_peak_sys, c='orange', label="sys. uncertainty")
plt.plot(time, E_peak-E_peak_sys, c='orange')
plt.hlines(np.mean(E_peak), time[0], time[-1], color='black', label="mean")
plt.xlabel("timestep")
plt.ylabel("energy [MeV]")
plt.title("peak energy over time")
plt.legend()

plt.show()
#plt.savefig(fname="temp", facecolor="white", dpi=100)

## Peak Energy movement

In [ ]:
time = range(20000, 120000+1, 2000)
peak_energy = 244.5             # peak energy for masks
Delta_t = 1.706e-16/1.28631

# start reading openPMD data
series = io.Series(path, io.Access.read_only)

# return id's of particles, which have peak energy at the last timestep
par_end = particlePMD(series.iterations[time[-1]].particles["b_all"])
E_end = par_end.Energy()
id_peak = par_end.particleId[(E_end>peak_energy-3)&(E_end<peak_energy+3)]

for i, t in enumerate(time):
    par = particlePMD(series.iterations[t].particles["b_all"])

    E = par.Energy()

    par.y_pos = par.y_pos - Delta_t * constants.c * t * 1e6 - 100
    
    # size of viewing window
    y_min, y_max, z_min, z_max = window(0)
    y_min -= 100
    y_max -= 100
    
    # shows only particles with E in given intervall
    E_mask = (E>peak_energy-2)&(E<peak_energy+2)
    id_mask = np.isin(par.particleId, id_peak)
    
    y_peak = par.y_pos[E_mask]
    z_peak = par.z_pos[E_mask]
    weight_peak = par.weight[E_mask]
    
    y_id_peak = par.y_pos[id_mask]
    z_id_peak = par.z_pos[id_mask]
    weight_id_peak = par.weight[id_mask]
    
    # plots
    plt.figure(figsize=(16, 5), dpi=70)
    
    # particle density plot where particles with peak density are highlighted
    plt.subplot(121)
    plt.hist2d(par.y_pos, par.z_pos, bins=200,  weights=par.weight, cmap='hot_r', norm=LogNorm(vmin=5e4, vmax=2e6), range=[[y_min, y_max], [z_min, z_max]])
    plt.colorbar(label='density (all particles)')
    plt.hist2d(y_peak, z_peak, bins=200,  weights=weight_peak, cmap='cool_r', norm=LogNorm(vmin=3e4, vmax=1e6), range=[[y_min, y_max], [z_min, z_max]])
    plt.colorbar(label='density (240 MeV < E < 250 MeV)')
    plt.xlabel("$y-ct \, \mathrm{[\mu m]}$")
    plt.ylabel("$z \, \mathrm{[\mu m]}$")
    plt.title("bunch-density, timstep: {:1d}".format(t))    
    
    # particle density plot where particles with peak density at step 90.000 are highlighted
    plt.subplot(122)
    plt.hist2d(par.y_pos, par.z_pos, bins=200,  weights=par.weight, cmap='hot_r', norm=LogNorm(vmin=5e4, vmax=2e6), range=[[y_min, y_max], [z_min, z_max]])
    plt.colorbar(label='density (all particles)')
    plt.hist2d(y_id_peak, z_id_peak, bins=200,  weights=weight_id_peak, cmap='cool_r', norm=LogNorm(vmin=3e4, vmax=2e6), range=[[y_min, y_max], [z_min, z_max]])
    plt.colorbar(label='density (240 MeV < E < 250 MeV on step 90.000)')
    plt.xlabel("$y-ct \, \mathrm{[\mu m]}$")
    plt.ylabel("$z \, \mathrm{[\mu m]}$")
    plt.title("bunch-density, timstep: {:1d}".format(t))   

    #plt.savefig(fname="temp", facecolor="white")
    plt.show()    

del series

## Electromagnetic Probe-Fields

In [ ]:
series = io.Series(path, io.Access.read_only)
time = range(12000, 20000+1, 2000)

for i, t in enumerate(time):
    y_min, y_max, z_min, z_max = window(t)
    particle = particlePMD(series.iterations[t].particles["b_all"])


    hist_Eyz, bin_y_E, bin_z_E, rest_E = stats.binned_statistic_2d(particle.y_pos, particle.z_pos, (particle.x_probeE*1e-9, particle.y_probeE*1e-9, particle.z_probeE*1e-9), statistic='mean', bins=60, range=[[y_min, y_max], [z_min, z_max]])
    y_Eyz, z_Eyz = binned_to_grid(bin_y_E, bin_z_E)
    abs_Eyz = np.sqrt(hist_Eyz[0].T**2 + hist_Eyz[1].T**2 + hist_Eyz[2].T**2)
    
    hist_Exz, bin_x_E, bin_z_E, rest_E = stats.binned_statistic_2d(particle.x_pos, particle.z_pos, (particle.x_probeE*1e-9, particle.y_probeE*1e-9, particle.z_probeE*1e-9), statistic='mean', bins=60, range=[[z_min, z_max], [z_min, z_max]])
    x_Exz, z_Exz = binned_to_grid(bin_x_E, bin_z_E)
    abs_Exz = np.sqrt(hist_Exz[0].T**2 + hist_Exz[1].T**2 + hist_Exz[2].T**2)
    
    probe_B_mask = (particle.x_pos>0)
    hist_Byz, bin_y_B, bin_z_B, rest_B = stats.binned_statistic_2d(particle.y_pos[probe_B_mask], particle.z_pos[probe_B_mask], (particle.x_probeB[probe_B_mask], particle.y_probeB[probe_B_mask], particle.z_probeB[probe_B_mask]), statistic='mean', bins=60, range=[[y_min, y_max], [z_min, z_max]])
    x_Bxz, z_Bxz = binned_to_grid(bin_y_B, bin_z_B)
    abs_Byz = np.sqrt(hist_Byz[0].T**2 + hist_Byz[1].T**2 + hist_Byz[2].T**2)
    
    hist_Bxz, bin_x_B, bin_z_B, rest_B = stats.binned_statistic_2d(particle.x_pos, particle.z_pos, (particle.x_probeB, particle.y_probeB, particle.z_probeB), statistic='mean', bins=60, range=[[z_min, z_max], [z_min, z_max]])
    x_Bxz, z_Bxz = binned_to_grid(bin_x_B, bin_z_B)
    abs_Bxz = np.sqrt(hist_Bxz[0].T**2 + hist_Bxz[1].T**2 + hist_Bxz[2].T**2)


    plt.figure(dpi=60, figsize=(24, 12))
    plt.subplot(231)
    plt.hist2d(particle.y_pos, particle.z_pos, bins=200,  weights=particle.weight, cmap='hot_r', norm=LogNorm(), range=[[y_min, y_max], [z_min, z_max]])
    plt.colorbar(label='particle density')
    plt.xlabel("$y \, \mathrm{[\mu m]}$")
    plt.ylabel("$z \, \mathrm{[\mu m]}$")
    plt.title("bunch-density, timstep: {:1d}".format(t))
    
    plt.subplot(232)
    plt.streamplot(y_Eyz, z_Eyz, hist_Eyz[1].T, hist_Eyz[2].T, density=3, color=abs_Eyz, cmap='magma_r')
    plt.colorbar(label='|probeE| [GV/m]')
    plt.xlabel("$y \, \mathrm{[\mu m]}$")
    plt.ylabel("$z \, \mathrm{[\mu m]}$")
    plt.title("y-z probeE, timstep: {:1d}".format(t))
    
    plt.subplot(233)
    plt.streamplot(x_Exz, z_Exz, hist_Exz[0].T, hist_Exz[2].T, density=3, color=abs_Exz, cmap='magma_r')
    plt.colorbar(label='|probeE| [GV/m]')
    plt.xlabel("$x \, \mathrm{[\mu m]}$")
    plt.ylabel("$z \, \mathrm{[\mu m]}$")
    plt.title("x-z probeE, timstep: {:1d}".format(t))
    
    plt.subplot(235)
    plt.streamplot(x_Bxz, z_Bxz, hist_Byz[1].T, hist_Byz[2].T, density=3, color=abs_Byz, cmap='magma_r')
    plt.colorbar(label='|probeB| [T]')
    plt.xlabel("$y \, \mathrm{[\mu m]}$")
    plt.ylabel("$z \, \mathrm{[\mu m]}$")
    plt.title("y-z probeB, timstep: {:1d}".format(t))
                    
    plt.subplot(236)
    plt.streamplot(x_Bxz, z_Bxz, hist_Bxz[0].T, hist_Bxz[2].T, density=3, color=abs_Bxz, cmap='magma_r')
    plt.colorbar(label='|probeB| [T]')
    plt.xlabel("$x \, \mathrm{[\mu m]}$")
    plt.ylabel("$z \, \mathrm{[\mu m]}$")
    plt.title("x-z probeB, timstep: {:1d}".format(t))
    
    plt.show()
#    plt.savefig(fname="temp", facecolor="white", dpi=300)
del series

## Lorentz-Force

In [ ]:
series = io.Series(path, io.Access.read_only)
time = range(20000, 160000+1, 2000)

for i, t in enumerate(time):
    y_min, y_max, z_min, z_max = window(t)
    particle = particlePMD(series.iterations[t].particles["b_all"])

    F_x_L, F_y_L, F_z_L = particle.LorentzForce()
    
    print("Sum of:\n  F_x = {:.4f} N\n  F_y = {:.4f} N\n  F_z = {:.4f} N".format(np.sum(F_x_L), np.sum(F_y_L), np.sum(F_z_L)))
    
    hist_Fyz, bin_y, bin_z, rest = stats.binned_statistic_2d(particle.y_pos, particle.z_pos, (F_x_L, F_y_L, F_z_L), statistic='mean', bins=60, range=[[y_min, y_max], [z_min, z_max]])
    y_Fyz, z_Fyz = binned_to_grid(bin_y, bin_z)
    abs_Fyz = np.sqrt(hist_Fyz[0].T**2 + hist_Fyz[1].T**2 + hist_Fyz[2].T**2)*1e3
    
    hist_Fxz, bin_x, bin_z, rest = stats.binned_statistic_2d(particle.x_pos, particle.z_pos, (F_x_L, F_y_L, F_z_L), statistic='mean', bins=60, range=[[z_min, z_max], [z_min, z_max]])
    x_Fxz, z_Fxz = binned_to_grid(bin_x, bin_z)
    abs_Fxz = np.sqrt(hist_Fxz[0].T**2 + hist_Fxz[1].T**2 + hist_Fxz[2].T**2)*1e3
    
    plt.figure(dpi=60, figsize=(24, 6))
    
    plt.subplot(131)
    plt.hist2d(particle.y_pos, particle.z_pos, bins=200,  weights=particle.weight, cmap='hot_r', norm=LogNorm(), range=[[y_min, y_max], [z_min, z_max]])
    plt.colorbar(label='particle density')
    plt.streamplot(y_Fyz, z_Fyz, hist_Fyz[1].T, hist_Fyz[2].T, density=2, color=abs_Fyz, cmap='Greens')
    plt.colorbar(label='$F_{Lorentz} \, \mathrm{[mN]}$')
    plt.xlabel("$y \, \mathrm{[\mu m]}$")
    plt.ylabel("$z \, \mathrm{[\mu m]}$")
    plt.title("bunch-density, timstep: {:1d}".format(t))
    
    plt.subplot(132)
    plt.streamplot(y_Fyz, z_Fyz, hist_Fyz[1].T, hist_Fyz[2].T, density=3, color=abs_Fxz, cmap='magma_r')
    plt.colorbar(label='$F_{Lorentz} \, \mathrm{[mN]}$')
    plt.xlabel("$y \, \mathrm{[\mu m]}$")
    plt.ylabel("$z \, \mathrm{[\mu m]}$")
    plt.title("y-z Lorentz-Force, timstep: {:1d}".format(t))
    
    plt.subplot(133)
    plt.streamplot(x_Fxz, z_Fxz, hist_Fxz[0].T, hist_Fxz[2].T, density=3, color=abs_Fxz, cmap='magma_r')
    plt.colorbar(label='$F_{Lorentz} \, \mathrm{[mN]}$')
    plt.xlabel("$x \, \mathrm{[\mu m]}$")
    plt.ylabel("$z \, \mathrm{[\mu m]}$")
    plt.title("x-z Lorentz-Force, timstep: {:1d}".format(t))

    plt.show()
#    plt.savefig(fname="temp", facecolor="white", dpi=300)
del series
del particle

## y-Lorentz-Force

In [ ]:
series = io.Series(path, io.Access.read_only)
time = range(20000, 170000+1, 2000)
bins=201
dQ = constants.e * 1e12

y_hist = np.zeros((len(time), bins))
E_end_hist = np.zeros((len(time), bins))
F_y_hist = np.zeros((len(time), bins)) 

# get y_pos at last timestep to show how particles changed position
par_end = particlePMD(series.iterations[time[-1]].particles["b_all"])
E_end = par_end.Energy()

par_beg = particlePMD(series.iterations[time[0]].particles["b_all"])

for i, t in enumerate(time):
    par = particlePMD(series.iterations[t].particles["b_all"])

    # size of viewing window
    y_min, y_max, z_min, z_max = window(t)
    
    dy = (y_max - y_min) / bins
    dz = (z_max - z_min) / bins
    
    # converting momenta to velocity
    F_x_L, F_y_L, F_z_L = par.LorentzForce()

    print("Min of F_y_L:", np.min(F_y_L), "mN")
    print("Min of F_z_L:", np.min(F_z_L), "mN")
    
    E = par.Energy()
    mask_end = np.isin(par.particleId, par_end.particleId)
    ind1 = np.argsort(par.particleId[mask_end])
    ind_end = np.argsort(par_end.particleId)
    
    mask_beg = np.isin(par_beg.particleId, par.particleId)
    ind2 = np.argsort(par.particleId)
    ind_beg = np.argsort(par_beg.particleId[mask_beg])

    hist_Fy, hist_Fz, hist_E = stats.binned_statistic_2d(par.y_pos, par.z_pos, (F_y_L, F_z_L, E), statistic='mean', bins=bins, range=[[y_min, y_max], [z_min, z_max]])[0] 
    hist_y_pos, hist_E_end = stats.binned_statistic_2d((par.y_pos[mask_end])[ind1], (par.z_pos[mask_end])[ind1], (par_end.y_pos[ind_end], E_end[ind_end]), statistic='min', bins=bins, range=[[y_min, y_max], [z_min, z_max]])[0]
    hist_z_pos = stats.binned_statistic_2d(par.y_pos[ind2], par.z_pos[ind2], ((par_beg.z_pos[mask_beg])[ind_beg]), statistic='mean', bins=bins, range=[[y_min, y_max], [z_min, z_max]])[0]

    y_hist[i] = hist_y_pos[:, 100]
    E_end_hist[i] = hist_E_end[:, 100]
    F_y_hist[i] = hist_Fy[:, 100]
    
    plot = True
    
    if plot:
        plt.figure(dpi=60, figsize=(24, 12))

        plt.subplot(231)
        plt.hist2d(par.y_pos, par.z_pos, bins=200,  weights=par.weight, cmap='hot_r', norm=LogNorm(vmin=5e4, vmax=3e6), range=[[y_min, y_max], [z_min, z_max]])
        plt.colorbar(label='particle density')
        plt.xlabel("$y \, \mathrm{[\mu m]}$")
        plt.ylabel("$z \, \mathrm{[\mu m]}$")
        plt.title("bunch-density, timstep: {:1d}".format(t))

        plt.subplot(232)
        plt.imshow(hist_Fy.T, cmap='seismic', extent=(y_min, y_max, z_min, z_max), aspect='auto', origin="lower", interpolation='none', norm=SymLogNorm(linthresh=5e-2, vmin=-1.5, vmax=1.5))
        plt.colorbar(label='$F_{Lorentz, y} \, \mathrm{[mN]}$')
        plt.xlabel("$y \, \mathrm{[\mu m]}$")
        plt.ylabel("$z \, \mathrm{[\mu m]}$")
        plt.title("y-Lorentz-Force, timstep: {:1d}".format(t))

        plt.subplot(233)
        plt.imshow(hist_Fz.T, cmap='seismic', extent=(y_min, y_max, z_min, z_max), aspect='auto', origin="lower", interpolation='none', norm=SymLogNorm(linthresh=5e-2, vmin=-1.5, vmax=1.5))
        plt.colorbar(label='$F_{Lorentz, z} \, \mathrm{[mN]}$')
        plt.xlabel("$y \, \mathrm{[\mu m]}$")
        plt.ylabel("$z \, \mathrm{[\mu m]}$")
        plt.title("z-Lorentz-Force, timstep: {:1d}".format(t))

        plt.subplot(235)
        plt.imshow(hist_y_pos.T, cmap='plasma', extent=(y_min, y_max, z_min, z_max), aspect='auto', vmin=np.min(par_end.y_pos), vmax=np.max(par_end.y_pos), origin="lower", interpolation='none')
        plt.colorbar(location='bottom').set_ticks([])
        plt.xlabel("$y \, \mathrm{[\mu m]}$")
        plt.ylabel("$z \, \mathrm{[\mu m]}$")
        plt.title("y-Postion at step 90.000, timstep: {:1d}".format(t))

        plt.subplot(236)
        plt.imshow(hist_z_pos.T, cmap='Spectral', extent=(y_min, y_max, z_min, z_max), aspect='auto', vmin=np.min(par_beg.z_pos), vmax=np.max(par_beg.z_pos), origin="lower", interpolation='none')
        plt.colorbar(label='$z_{0} \, \mathrm{[\mu m]}$')
        plt.xlabel("$y \, \mathrm{[\mu m]}$")
        plt.ylabel("$z \, \mathrm{[\mu m]}$")
        plt.title("z-Postion at step 0, timstep: {:1d}".format(t))  

        plt.subplot(234)
        plt.imshow(hist_E.T, cmap='jet', vmin=0, vmax=300, extent=(y_min, y_max, z_min, z_max), aspect='auto', origin="lower", interpolation='none')
        plt.colorbar(label='$E \, \mathrm{[meV]}$')
        plt.xlabel("$y \, \mathrm{[\mu m]}$")
        plt.ylabel("$z \, \mathrm{[\mu m]}$")
        plt.title("kinetic Energy, timstep: {:1d}".format(t))

        plt.show()
#        plt.savefig(fname="temp", facecolor="white", dpi=300)
    else:
        print("timestep: ", t)

del series


y-position arguments over time

In [ ]:
y_min, y_max, z_min, z_max = window(0)

plt.figure(dpi=60, figsize=(24, 6))
plt.subplot(131)
plt.imshow(y_hist, cmap="plasma", origin="lower", aspect='auto', vmin=np.min(par_end.ypos), vmax=np.max(par_end.ypos), extent=(y_min, y_max, time[0]-1e3, time[-1]+1e3), interpolation='none')
plt.colorbar(label="$y_{90 000} \, \mathrm{[\mu m]}$")
plt.xlabel("$y-c\cdot t \, \mathrm{[\mu m]}$")
plt.ylabel("$timestep$")

plt.subplot(132)
plt.imshow(E_end_hist, cmap="jet", origin="lower", aspect='auto', vmin=np.min(0), vmax=np.max(300), extent=(y_min, y_max, time[0]-1e3, time[-1]+1e3), interpolation='none')
plt.colorbar(label="$E_{120 000} \, \mathrm{[MeV]}$")
plt.xlabel("$y-c\cdot t \, \mathrm{[\mu m]}$")
plt.ylabel("$timestep$")

plt.subplot(133)
plt.imshow(F_y_hist, cmap="seismic", origin="lower", aspect='auto', vmin=-1, vmax=1, extent=(y_min, y_max, time[0]-1e3, time[-1]+1e3), interpolation='none')
plt.colorbar(label="$F_y \, \mathrm{[mN]}$")
plt.xlabel("$y-c\cdot t \, \mathrm{[\mu m]}$")
plt.ylabel("$timestep$")
plt.show()


## Test Multidimensional color

In [ ]:
series = io.Series(path, io.Access.read_only)
time = range(16000, 30000+1, 2000)

vmin = 5e4
vmax = 3e6

map_h = np.linspace(2/3, 0, 200)
map_s = (np.log10(np.linspace(vmin, vmax, 200)) - np.log10(vmin)) / (np.log10(vmax) - np.log10(vmin))
map_hs = np.meshgrid(map_h, map_s)
map_v = np.ones((200, 200))

maphsv = np.dstack((map_hs[0], map_hs[1], map_v))
maprgb = np.transpose(hsv_to_rgb(maphsv), axes=(1,0,2))


for i, t in enumerate(time):
    particle = series.iterations[t].particles["b_all"]
    
    weight, x_pos, y_pos, z_pos, x_mom, y_mom, z_mom = particlePMD(particle)
    
    E = mom_to_E(x_mom, y_mom, z_mom, weight)
    
    plt.figure(dpi=60, figsize=(16, 6))

    statistic, bin_y, bin_z, rest = stats.binned_statistic_2d(y_pos, z_pos, (E), statistic='mean', bins=200)
    statistic2, bin_y2, bin_z2, rest2 = stats.binned_statistic_2d(y_pos, z_pos, (weight), statistic='sum', bins=200)
    
    hue  = (-(np.clip(statistic, 0, 300) / 300)+1)*2/3
    saturation = (np.log10(np.clip(statistic2, vmin, vmax)) - np.log10(vmin)) / (np.log10(vmax) - np.log10(vmin))
    saturation[statistic2==0] = 0
#    print(saturation[100])
#    print(statistic2[100])
    value = np.ones_like(statistic)
    
    hsv = np.dstack((hue, saturation, value))
    rgb = np.transpose(hsv_to_rgb(hsv), axes=(1,0,2))
    

    plt.subplot(121)
    plt.imshow(rgb, extent=(bin_y[0], bin_y[-1], bin_z[0], bin_z[-1]), aspect='auto', origin="lower", interpolation='none')
    plt.xlabel("$y \, \mathrm{[\mu m]}$")
    plt.ylabel("$z \, \mathrm{[\mu m]}$")
    plt.title("kinetic Energy, timstep: {:1d}".format(t))
        
    plt.subplot(122)
    plt.imshow(maprgb, origin="lower", aspect='auto', extent=(5e4, 3e6, 0, 300))
    plt.xlabel("$density$")
    plt.ylabel("$Energy \, \mathrm{[MeV]}$")
    plt.xscale('log')

    plt.show()
#    plt.savefig(fname="temp", facecolor="white", dpi=300)
del series

# energy muss geklippt werden bei 300 MeV
# Transponieren
# Sättigung logarithmisch

In [ ]:
series = io.Series(path, io.Access.read_only)
time = range(16000, 40000+1, 2000)

vmin = 5e4
vmax = 3e6

map_h = np.linspace(2/3, 0, 200)
map_s = np.ones(200)
map_v = np.ones(200)
map_a = (np.log10(np.linspace(vmin, vmax, 200)) - np.log10(vmin)) / (np.log10(vmax) - np.log10(vmin))

map_ha = np.meshgrid(map_h, map_a)
# How to mesh alpha and colors?

maphsv = np.dstack((map_h, map_s, map_v))
maprgb = hsv_to_rgb(maphsv)
maprgba = np.meshgrid(maprgb, map_a)[1]
#np.transpose(hsv_to_rgb(maphsv), axes=(1,0,2))
print(np.meshgrid(maprgb, map_a)[0])


for i, t in enumerate(time):
    particle = series.iterations[t].particles["b_all"]
    
    weight, x_pos, y_pos, z_pos, x_mom, y_mom, z_mom = particlePMD(particle)
    
    E = mom_to_E(x_mom, y_mom, z_mom, weight)
    
    plt.figure(dpi=60, figsize=(16, 6))

    statistic, bin_y, bin_z, rest = stats.binned_statistic_2d(y_pos, z_pos, (E), statistic='mean', bins=200)
    statistic2, bin_y2, bin_z2, rest2 = stats.binned_statistic_2d(y_pos, z_pos, (weight), statistic='sum', bins=200)
        
    statistic = np.nan_to_num(statistic)
    hue = (-(np.clip(statistic, 0, 300) / 300) + 1) * 2/3
    saturation = np.ones_like(statistic)
    value = np.ones_like(statistic)
    
    alpha = (np.log10(np.clip(statistic2, vmin, vmax)) - np.log10(vmin)) / (np.log10(vmax) - np.log10(vmin))
    alpha[statistic2==0] = 0
    
    hsv = np.dstack((hue, saturation, value))
    rgb = hsv_to_rgb(hsv)
    rgba = np.transpose(np.dstack((rgb, alpha)), axes=(1,0,2))

    plt.subplot(121)
    plt.imshow(rgba, extent=(bin_y[0], bin_y[-1], bin_z[0], bin_z[-1]), aspect='auto', origin="lower", interpolation='none')
    plt.xlabel("$y \, \mathrm{[\mu m]}$")
    plt.ylabel("$z \, \mathrm{[\mu m]}$")
    plt.title("kinetic Energy, timstep: {:1d}".format(t))
        
    plt.subplot(122)
    plt.imshow(maprgba, origin="lower", aspect='auto', extent=(5e4, 3e6, 0, 300))
    plt.xlabel("$density$")
    plt.ylabel("$Energy \, \mathrm{[MeV]}$")
    plt.xscale('log')

    plt.show()
#    plt.savefig(fname="temp", facecolor="white", dpi=300)
del series

# energy muss geklippt werden bei 300 MeV
# Transponieren
# Sättigung logarithmisch

In [ ]:
div_max = 50
weight = np.ones(int(5e4), dtype=np.int32) * 5e4

E1_gauss = np.random.normal(loc=245, scale=5, size=int(3e4))
div1_gauss = np.random.normal(loc=0, scale=8, size=int(3e4))

E2_gauss = np.random.normal(loc=208, scale=20, size=int(2e4))
div2_gauss = np.random.normal(loc=0, scale=20, size=int(2e4))

E_gauss = np.append(E1_gauss, [E2_gauss])
div_gauss = np.append(div1_gauss, [div2_gauss])
h, x_edge, y_edge, r = plt.hist2d(E_gauss, div_gauss, weights=weight, bins=[1000, 201], cmap="hot_r", range=[[0, 400], [-div_max, div_max]], norm=LogNorm(vmin=1e4))
plt.colorbar()

plt.show()

x = y_edge[:-1]
xwidth = x[1] - x[0]
x += xwidth/2
y = np.sum(h, axis=0)

p0 = [8e7, 0, 5]
parameters, covariance = curve_fit(Gauss, x, y, p0)
fit_a = parameters[0]
fit_b = parameters[1]
fit_c = parameters[2]
fit_y = Gauss(x, fit_a, fit_b, fit_c)

plt.bar(x, y)
plt.plot(x, fit_y, label='fit', c='orange')

plt.show()
#E3_gauss = Gauss3(xx, 160, 20, 0.15e6)
#div3_gauss = Gauss3(yy, 0, 30, 1)


In [ ]:
time = range(12000, 30000+1, 2000)

div = np.zeros(len(time))
series = io.Series(path, io.Access.read_only)
    
for i, t in enumerate(time):
    par = particlePMD(series.iterations[t].particles["b_all"])
    y_min, y_max, z_min, z_max = window(t)

    div[i] = np.std(np.arctan2(par.z_mom, par.y_mom))*1e3

del series
plt.figure(figsize=(8, 4))
plt.plot(time, div, marker='.')
plt.xlabel("$timestep$")
plt.ylabel("$divergence \, \mathrm{[mrad]}$")
plt.title("standard deviation of deviation in vacuum propagation")


#plt.savefig(fname="temp", facecolor="white", dpi=100)
plt.show()